# Problem 1a

In [231]:
## Setup
# load packages
import numpy as np
import csv
import random
import math

# load data
with open("Concrete_Data_Yeh.csv") as fp:
    f = csv.reader(fp, delimiter=",", quotechar='"')
    conc_dat = np.array([row for row in f])

# add intercept term to features
conc_dat = np.concatenate((np.ones((conc_dat.shape[0], 1)), conc_dat), axis=1)
conc_dat[0,0] = "(intercept)"

# split out header
header = conc_dat[0,:]

In [66]:
## Functions
def solve_beta(data):
    """Estimates beta coefficients via numpy."""
  
    # separate into features vs. outputs

    header = data[0]
    y_ind = np.where(header == "csMPa")[0][0]

    X = np.array(X0[1:, :y_ind], dtype="float")
    Y = np.array(X0[1:,y_ind], dtype="float")

    # solve for (X^T*X)^-1
    XtX_inv = np.linalg.inv(np.dot(X.T, X))

    # solve for beta
    beta = np.dot(np.dot(XtX_inv, X.T), Y)
    return beta, X, Y

# Problem 1b

In [240]:
## Split into training/validation sets (75/25)
# make vector of indices
inds = [i + 1 for i in list(range(len(conc_dat)-1))]

# shuffle randomly
random.shuffle(inds)

# take first 75%/last 25%
train_inds = inds[:math.floor(0.75*len(conc_dat))]
val_inds = inds[math.floor(0.75*len(conc_dat)):]
train = np.array([conc_dat[i] for i in train_inds])
val = np.array([conc_dat[i] for i in val_inds])

# add header to training datasets
train = np.vstack((header, train))

## Build alternative models (first 7/8/9 columns)
## Fit models to training data 
m1, _, _ = solve_beta(train[:, np.array([0,1,2,3,4,5,9])])
m2, _, _ = solve_beta(train[:, np.array([0,1,2,3,4,5,6,9])])
m3, _, _ = solve_beta(train[:, np.array([0,1,2,3,4,5,6,9])])

## Compute MSE with test data

y_ind = np.where(header == "csMPa")[0][0]
Y_val = np.array(val[:,y_ind], dtype="float")
val = np.array(val[0:], dtype="float")
val = np.concatenate((np.ones((val.shape[0], 1)), val), axis=1)

mse1 = sum((np.dot(val[:, np.array([0,1,2,3,4,5,6])], m1) - Y_val)**2/2)
mse2 = sum((np.dot(val[:, np.array([0,1,2,3,4,5,6,7])], m2) - Y_val)**2/2)
mse3 = sum((np.dot(val[:, np.array([0,1,2,3,4,5,6,7,8])], m3) - Y_val)**2/2)

print(mse1)
print(mse2)
print(mse3)

#### something is wrong with the indexing for val ####
#### need to step through solve_beta with val ####

LinAlgError: Singular matrix

1